# Exploring and Cleaning Returns Dataset

In this notebook, we will be exploring the Chinese equities returns dataset.

In [1]:
#!pip install pyreadr

     |████████████████████████████████| 256kB 5.4MB/s eta 0:00:01


In [2]:
# Import the packages we will need
import numpy as np
import pandas as pd
import os

# Used to read .rds file
import pyreadr

In [5]:
# Read the Data
data = pyreadr.read_r('../../dailyChina.Rds')
data = data[None]

In [6]:
data.head()

,id,prc,Date,ret,lagME,industryCitic
0,000001,79.614491,2007-02-01,-0.035024,2.977886e+10,Banks
1,000001,75.646705,2007-02-02,-0.049837,2.873590e+10,Banks
2,000001,71.980815,2007-02-05,-0.048461,2.730378e+10,Banks
3,000001,75.560448,2007-02-06,0.049730,2.598062e+10,Banks
4,000001,79.355722,2007-02-07,0.050228,2.727264e+10,Banks


In a meeting, I was told that:
* id -- The ID of the company on the exchange
* prc -- The closing stock price of the company
* Date -- Self-explanatory
* ret -- Daily return for the stock
* lagME -- Market capitalization of the stock
* industryCitic -- Sector of the company

We will need to do some data wrangling to create an index return dataframe. Let's get started:

In [8]:
# Extract one company's data
sample = data.loc[data['id'] == '000001']
sample

,id,prc,Date,ret,lagME,industryCitic
0,000001,79.614491,2007-02-01,-0.035024,2.977886e+10,Banks
1,000001,75.646705,2007-02-02,-0.049837,2.873590e+10,Banks
2,000001,71.980815,2007-02-05,-0.048461,2.730378e+10,Banks
3,000001,75.560448,2007-02-06,0.049730,2.598062e+10,Banks
4,000001,79.355722,2007-02-07,0.050228,2.727264e+10,Banks
...,...,...,...,...,...,...
2553972,000001,13.450000,2020-04-21,0.035412,1.260415e+11,Banks
2553973,000001,13.230000,2020-04-22,-0.016357,1.305049e+11,Banks
2553974,000001,13.230000,2020-04-23,0.000000,1.283703e+11,Banks
2553975,000001,13.240000,2020-04-24,0.000756,1.283703e+11,Banks


In [9]:
# We want the Date and returns only for now
sample_returns = sample.loc[:,['Date', 'ret']].set_index('Date')
sample_returns

,ret
Date,
2007-02-01,-0.035024
2007-02-02,-0.049837
2007-02-05,-0.048461
2007-02-06,0.049730
2007-02-07,0.050228
...,...
2020-04-21,0.035412
2020-04-22,-0.016357
2020-04-23,0.000000


This seems simple enough, let's try to put this altogether now:

In [10]:
# Create a dictionary as it's easy to concatenate afterwards
returns_dict = {}

# Loop through each unque id
for ID in set(data['id'].values):
    
    # Same as before
    sample = data.loc[data['id'] == ID]
    sample_returns = sample.loc[:,['Date', 'ret']].set_index('Date')
    sample_returns.columns = [ID]
    returns_dict[ID] = sample_returns.copy()

In [13]:
# Sanity check
returns_dict['002110']

,002110
Date,
2007-08-01,-0.067320
2007-08-02,0.025928
2007-08-03,0.013661
2007-08-06,0.037736
2007-08-07,0.033766
...,...
2020-04-21,-0.016086
2020-04-22,-0.019074
2020-04-23,-0.005556


Looks good! Let's concatenate all the returns now into a single dataframe:

In [14]:
returns = pd.concat([x for x in returns_dict.values()], axis=1)
returns

,600525,300274,601928,600717,603160,600651,600160,600378,601068,600469,...,600692,002841,600592,002064,600064,002047,600699,600597,000501,600051
Date,,,,,,,,,,,,,,,,,,,,,
2007-02-01,-0.003488,NaN,NaN,-0.018416,NaN,NaN,0.019264,-0.008333,NaN,0.091954,...,0.029279,NaN,0.007645,NaN,-0.006563,-0.001653,NaN,0.003968,0.050407,0.001119
2007-02-02,-0.000269,NaN,NaN,-0.000938,NaN,NaN,-0.015464,0.021008,NaN,-0.025000,...,-0.030635,NaN,-0.005311,NaN,0.000000,-0.014901,NaN,-0.046113,0.054180,-0.032402
2007-02-05,0.002155,NaN,NaN,0.008451,NaN,NaN,0.036649,0.047325,NaN,0.000000,...,0.011287,NaN,0.004577,NaN,-0.069364,0.000000,NaN,-0.001381,0.099853,0.013857
2007-02-06,0.030099,NaN,NaN,0.022346,NaN,NaN,0.035354,-0.055010,NaN,0.048583,...,0.037946,NaN,0.010630,NaN,0.006211,0.057143,NaN,0.040111,0.029373,0.055809
2007-02-07,0.045656,NaN,NaN,0.029144,NaN,NaN,0.011382,0.035343,NaN,0.014157,...,0.002151,NaN,0.015778,NaN,0.029982,-0.011129,NaN,0.003989,0.058366,-0.007551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-19,NaN,NaN,NaN,0.007194,NaN,-0.004049,0.010163,NaN,NaN,0.043546,...,0.010101,NaN,0.013769,0.008824,0.047909,NaN,NaN,-0.014644,0.010490,0.004792
2009-01-20,NaN,NaN,NaN,0.009184,NaN,0.008130,-0.016097,NaN,NaN,0.034277,...,-0.011667,NaN,-0.008489,-0.007289,0.013300,NaN,NaN,0.004246,0.015571,-0.007949
2009-01-21,NaN,NaN,NaN,0.010111,NaN,-0.006048,0.010225,NaN,NaN,-0.021614,...,-0.011804,NaN,0.041096,0.036711,0.035275,NaN,NaN,-0.021142,-0.018739,-0.003205


This looks very promising! Let's now do a sanity check to ensure all our steps were done correctly:

In [16]:
all(data.loc[data['id'] == '002110']['ret'].values == returns['002110'].dropna().values)

False

Great! Let's export this now to an h5 file:

In [17]:
returns.to_hdf('returns.h5', 'returns')